# Add Image Metrics to an existing Table

In [2]:
from pathlib import Path

DATA_PATH = Path("../data/coco128/images").absolute().as_posix()


## Write the initial table

In [3]:
import os
import tlc

table_writer = tlc.TableWriter(
    table_name="coco128",
    dataset_name="coco128",
    project_name="add-image-metrics",
    description="COCO128 dataset",
    column_schemas={
        "image": tlc.ImagePath
    }
)

for image_name in os.listdir(DATA_PATH):
    image_path = os.path.join(DATA_PATH, image_name)
    table_writer.add_row({"image": image_path})

table = table_writer.finalize()


## Extend the table with image metrics

In [15]:
from PIL import Image, ImageStat
import numpy as np

def compute_image_metrics(image_path: str):    
    """Return a dict of image metrics for the given image path."""
    image = Image.open(image_path)
    width, height = image.size
    pixels = np.array(image)

    # Convert to grayscale for some metrics
    grayscale_image = image.convert("L")
    stat = ImageStat.Stat(grayscale_image)

    # Compute brightness (average grayscale value)
    brightness = stat.mean[0]

    # Compute contrast (standard deviation of grayscale values)
    contrast = stat.stddev[0]

    # Compute average RGB values
    try:
        avg_r = np.mean(pixels[:, :, 0])
        avg_g = np.mean(pixels[:, :, 1])
        avg_b = np.mean(pixels[:, :, 2])
    except IndexError: # Image is grayscale
        avg_r = avg_g = avg_b = 0

    return {
        "width": width,
        "height": height,
        "brightness": brightness,
        "contrast": contrast,
        "average_red": avg_r,
        "average_green": avg_g,
        "average_blue": avg_b,
    }

In [24]:
extended_table_writer = tlc.TableWriter(
    table_name="added-image-metrics",
    dataset_name="coco128",
    project_name="add-image-metrics",
    description="COCO128 dataset with added image metrics",
    column_schemas={
        "image": tlc.ImagePath,
        "width": tlc.Schema(value=tlc.Int32Value(), writable=False, sample_type="hidden"),
        "height": tlc.Schema(value=tlc.Int32Value(), writable=False, sample_type="hidden"),
        "brightness": tlc.Schema(value=tlc.Float32Value(), writable=False, sample_type="hidden"),
        "contrast": tlc.Schema(value=tlc.Float32Value(), writable=False, sample_type="hidden"),
        "average_red": tlc.Schema(value=tlc.Float32Value(), writable=False, sample_type="hidden"),
        "average_green": tlc.Schema(value=tlc.Float32Value(), writable=False, sample_type="hidden"),
        "average_blue": tlc.Schema(value=tlc.Float32Value(), writable=False, sample_type="hidden"),
    },
)

for row in table:
    image_path = row["image"]
    metrics = compute_image_metrics(image_path)
    new_row = {**row, **metrics}
    extended_table_writer.add_row(new_row)

extended_table = extended_table_writer.finalize()

print(extended_table[0])

{'image': 'C:/Project/notebook-examples/data/coco128/images/000000000009.jpg'}
